In [ ]:
# default_exp benchmark.traceability

# Traceability Benchmark

> This module is dedicated benchmarking

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
from nbdev.export import notebook2script
notebook2script()